<a href="https://colab.research.google.com/github/LokeRuiKee/AChE-GNN/blob/main/SMILES_To_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://www.blopig.com/blog/2022/02/how-to-turn-a-smiles-string-into-a-molecular-graph-for-pytorch-geometric/

# Step 0: Install and Import Packages

In [ ]:
!pip install rdkit
!pip install torch
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00


In [ ]:
# import packages

# general tools
import numpy as np

# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

# Step 1: Atom Featurisation

In [ ]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding

In [ ]:
def get_atom_features(atom,
                      use_chirality = True,
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """

    # define list of permitted atoms

    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']

    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms

    # compute atom features

    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)

    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])

    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])

    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])

    is_in_a_ring_enc = [int(atom.IsInRing())]

    is_aromatic_enc = [int(atom.GetIsAromatic())]

    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]

    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]

    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled

    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc

    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

# Step 2: Bond Featurisation

In [ ]:
def get_bond_features(bond,
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)

    bond_is_conj_enc = [int(bond.GetIsConjugated())]

    bond_is_in_ring_enc = [int(bond.IsInRing())]

    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc

    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

# Step 3: Generating labeled Pytorch Geometric Graph Objects

In [ ]:
import pandas as pd

def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(file_path, smiles_column_name, y_column_name):

    data_list = []

    # Read the Excel file
    df = pd.read_excel(file_path)

    # Extract SMILES and labels from the specified columns
    x_smiles = df[smiles_column_name].tolist()
    y = df[y_column_name].tolist()

    for (smiles, y_val) in zip(x_smiles, y):

        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)

        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)

        X = torch.tensor(X, dtype = torch.float)

        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)

        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))

        for (k, (i,j)) in enumerate(zip(rows, cols)):

            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))

        EF = torch.tensor(EF, dtype = torch.float)

        # construct label tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)

        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))

    return data_list

# test
file_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/tx2c00283_si_002/SupplementalFiles/Human_dataset_1micromolar.xlsx"
smiles_column_name = "SMILES"  # Adjust this to the actual column name in your Excel file
y_column_name = "single-class-label"  # Adjust this to the actual column name in your Excel file

data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(file_path, smiles_column_name, y_column_name)


# Training Loop

In [ ]:
from torch.nn import Linear, ReLU
import torch_geometric.nn as nn
from torch_geometric.nn import Sequential, GCNConv, JumpingKnowledge

In [ ]:
from torch_geometric.nn import GCN, summary

model = GCN(128, 64, num_layers=2, out_channels=32)
x = torch.randn(100, 128)
edge_index = torch.randint(100, size=(2, 20))

print(summary(model, x, edge_index))

+---------------------+---------------------+----------------+----------+
| Layer               | Input Shape         | Output Shape   | #Param   |
|---------------------+---------------------+----------------+----------|
| GCN                 | [100, 128], [2, 20] | [100, 32]      | 10,336   |
| ├─(dropout)Dropout  | [100, 64]           | [100, 64]      | --       |
| ├─(act)ReLU         | [100, 64]           | [100, 64]      | --       |
| ├─(convs)ModuleList | --                  | --             | 10,336   |
| │    └─(0)GCNConv   | [100, 128], [2, 20] | [100, 64]      | 8,256    |
| │    └─(1)GCNConv   | [100, 64], [2, 20]  | [100, 32]      | 2,080    |
| ├─(norms)ModuleList | --                  | --             | --       |
| │    └─(0)Identity  | [100, 64]           | [100, 64]      | --       |
| │    └─(1)Identity  | --                  | --             | --       |
+---------------------+---------------------+----------------+----------+


In [ ]:
# canonical training loop for a Pytorch Geometric GNN model gnn_model

file_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/tx2c00283_si_002/SupplementalFiles/Human_dataset_1micromolar.xlsx"
smiles_column_name = "SMILES"
y_column_name = "single-class-label"

# create list of molecular graph objects from list of SMILES x_smiles and list of labels y
data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(file_path, smiles_column_name, y_column_name)

# create dataloader for training
dataloader = DataLoader(dataset = data_list, batch_size = 2**7)


# define loss function
loss_function = torch.nn.MSELoss()

# define optimiser
gnn_model = GCN(79, 128, 2)
optimiser = torch.optim.Adam(gnn_model.parameters(), lr = 1e-3)

# loop over 10 training epochs
for epoch in range(10):

    # set model to training mode
    gnn_model.train()

    # loop over minibatches for training
    for (k, batch) in enumerate(dataloader):

        # compute current value of loss function via forward pass
        output = gnn_model(batch)
        loss_function_value = loss_function(output[:,0], torch.tensor(batch.y, dtype = torch.float32))

        # set past gradient to zero
        optimiser.zero_grad()

        # compute current gradient via backward pass
        loss_function_value.backward()

        # update model weights using gradient and optimisation method
        optimiser.step()


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

In [ ]:
# canonical training loop for a Pytorch Geometric GNN model gnn_model

file_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/tx2c00283_si_002/SupplementalFiles/Human_dataset_1micromolar.xlsx"
smiles_column_name = "SMILES"
y_column_name = "single-class-label"

# create list of molecular graph objects from list of SMILES x_smiles and list of labels y
data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(file_path, smiles_column_name, y_column_name)

# create dataloader for training
dataloader = DataLoader(dataset = data_list, batch_size = 2**7)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.data.batch import Batch

# Define custom collate function
def custom_collate(batch):
    """
    Custom collate function to process Data objects.
    """
    # Extract individual components from Data objects
    x = [data.x for data in batch]
    edge_index = [data.edge_index for data in batch]
    edge_attr = [data.edge_attr for data in batch]
    y = [data.y for data in batch]

    # Convert list of Data objects to Batch object
    batch = Batch.from_data_list(batch)

    # Verify that edge indices are properly aligned with node features and edge attributes
    assert batch.edge_index.shape[1] == batch.edge_attr.shape[1], "Edge indices and edge attributes must have the same number of edges"

    # Ensure proper alignment of node features and edge attributes
    assert batch.edge_index[0].max() < batch.x.size(0) and batch.edge_index[1].max() < batch.x.size(0), "Invalid edge indices with respect to node features"
    assert batch.edge_index[0].max() < batch.edge_attr.size(0) and batch.edge_index[1].max() < batch.edge_attr.size(0), "Invalid edge indices with respect to edge attributes"

    return batch

# Create DataLoader using custom collate function
dataloader = DataLoader(dataset=data_list, batch_size=2**7, collate_fn=custom_collate)


In [ ]:
# define loss function
loss_function = torch.nn.MSELoss()

# define optimiser
gnn_model = GCN(79, 128, 2)
optimiser = torch.optim.Adam(gnn_model.parameters(), lr = 1e-3)

# loop over 10 training epochs
for epoch in range(10):

    # set model to training mode
    gnn_model.train()# loop over minibatches for training

    for (k, batch) in enumerate(dataloader):

        # Extract individual components from the batch
        x = batch.x  # Node features
        edge_index = batch.edge_index  # Edge indices
        edge_attr = batch.edge_attr  # Edge attributes
        y = batch.y  # Labels

        # Pass the extracted components to the GNN model
        output = gnn_model(x, edge_index, edge_attr)

        # Compute the current value of the loss function via forward pass
        loss_function_value = loss_function(output, y)

        # Set past gradient to zero
        optimiser.zero_grad()

        # Compute current gradient via backward pass
        loss_function_value.backward()

        # Update model weights using gradient and optimisation method
        optimiser.step()


AssertionError: Edge indices and edge attributes must have the same number of edges

In [ ]:
print(data_list)

[Data(x=[16, 79], edge_index=[2, 32], edge_attr=[32, 10], y=[1]), Data(x=[31, 79], edge_index=[2, 68], edge_attr=[68, 10], y=[1]), Data(x=[40, 79], edge_index=[2, 90], edge_attr=[90, 10], y=[1]), Data(x=[33, 79], edge_index=[2, 72], edge_attr=[72, 10], y=[1]), Data(x=[21, 79], edge_index=[2, 48], edge_attr=[48, 10], y=[1]), Data(x=[20, 79], edge_index=[2, 46], edge_attr=[46, 10], y=[1]), Data(x=[37, 79], edge_index=[2, 80], edge_attr=[80, 10], y=[1]), Data(x=[28, 79], edge_index=[2, 64], edge_attr=[64, 10], y=[1]), Data(x=[43, 79], edge_index=[2, 98], edge_attr=[98, 10], y=[1]), Data(x=[43, 79], edge_index=[2, 98], edge_attr=[98, 10], y=[1]), Data(x=[42, 79], edge_index=[2, 96], edge_attr=[96, 10], y=[1]), Data(x=[44, 79], edge_index=[2, 100], edge_attr=[100, 10], y=[1]), Data(x=[20, 79], edge_index=[2, 46], edge_attr=[46, 10], y=[1]), Data(x=[42, 79], edge_index=[2, 96], edge_attr=[96, 10], y=[1]), Data(x=[29, 79], edge_index=[2, 66], edge_attr=[66, 10], y=[1]), Data(x=[41, 79], edge_

In [ ]:
print(summary(GCN, x, edge_index))